In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Step 1: Rectangular Box Extraction
Applying classical image processing methods such as Canny filter, dilation and erotion and Houghes Transform to extract the main rectangular shapes from the image

In [32]:
from image_processing.BoxDetector import BroadBoxDetector
from image_processing.BoxDetectorUtils import BoxDetectorUtils
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
import numpy as np
import pandas as pd
import os
root = "../../assets/imgs"
individual_imgs_results_dir =  "../../results/individual_spots"

## Feature Extraction
We us the top layer, without the classsification head, of a vgg model to map the images to a feature space. 

In [33]:
import torch
# model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)

Using cache found in /home/mrtbuntu/.cache/torch/hub/pytorch_vision_v0.10.0


In [34]:
from PIL import Image
from torchvision import transforms
from functools import lru_cache

@lru_cache(500)
def extract_features(image_path):
    """ Given the image path, apply preprocessing and compute the feature vector"""
    input_image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    return output[0].cpu().detach().numpy()

In [35]:
images = os.listdir(root) # raw video thumbnails
results_df = pd.DataFrame(columns=["src_image","dst_image", "box", "feature_vector"])

for src_image in tqdm(images):
    image_path = f"{root}/{src_image}"
    pixels = plt.imread(image_path)
    BBD = BroadBoxDetector()
    bounding_boxes = BBD.detect_boxes(pixels, find_zocalos=False)

    for counter, bounding_box in enumerate(bounding_boxes):
        individual_spot = BoxDetectorUtils.crop_frame(pixels, bounding_box) # cajita individual
        individual_name = f"{src_image.split('.')[0]}_{counter}.jpg"

        individual_box_path = f"{individual_imgs_results_dir}/{individual_name}"
        plt.imsave(individual_box_path, individual_spot)

        vgg11_features = extract_features(individual_box_path)
        bgr_hist = BoxDetectorUtils.get_bgr_hist(individual_spot, bins=32)

        feature_vector = vgg11_features #np.concatenate((vgg11_features, bgr_hist))
        # feature_vector = np.concatenate((vgg11_features, bgr_hist))
        data = {
            "src_image":src_image,
            "dst_image":individual_name,
            "box": bounding_box,
            "feature_vector": feature_vector
        }
        results_df = results_df.append(data, ignore_index=True)

  0%|          | 0/326 [00:00<?, ?it/s]

In [36]:
# dataset_name = "results.pck"
dataset_name  = "results_vgg16BGR2.pck"

In [37]:
results_df.to_pickle(f"../../results/{dataset_name}")

## Postprocess dataset

In [38]:
results_df = pd.read_pickle(f"../../results/{dataset_name}")
results_df = results_df[results_df['src_image']!='betsson.jpeg']
src_image_column = results_df['src_image'].map(lambda name: int(name.split(".")[0]))
results_df['src_image_number'] = src_image_column

In [39]:
results_df

,src_image,dst_image,box,feature_vector,src_image_number
0,0.jpg,0_0.jpg,"(356, 327, 151, 104)","[-2.6547847, -0.8471976, -3.809546, -2.3656282...",0
1,0.jpg,0_1.jpg,"(202, 326, 151, 105)","[-2.5192688, -2.3182557, -3.2193217, -2.103216...",0
2,1.jpg,1_0.jpg,"(364, 331, 152, 103)","[-0.7869164, 0.036874715, -0.5789321, 0.159186...",1
3,1.jpg,1_1.jpg,"(193, 332, 152, 102)","[-1.8711119, -1.6433448, -1.005236, -0.1895348...",1
4,100.jpg,100_0.jpg,"(347, 301, 160, 108)","[-2.7626224, -0.7975287, -2.7019863, -3.088944...",100
...,...,...,...,...,...
515,96.jpg,96_0.jpg,"(187, 302, 160, 107)","[-1.4451208, -1.5924901, 0.5326917, -1.4767649...",96
516,96.jpg,96_1.jpg,"(363, 301, 160, 108)","[-1.9138888, -1.0630454, 0.69642, -1.2709609, ...",96
517,98.jpg,98_0.jpg,"(288, 284, 134, 91)","[-1.9513379, -1.0318924, -0.28411716, 1.256894...",98
518,99.jpg,99_0.jpg,"(363, 301, 160, 108)","[-0.72070426, -1.5725075, -1.9164596, -1.16294...",99


# Clustering y clasificación

Una vez obtenidos los feature vectors podemos correr un algoritmo de clasificación que nos permita agruparlas por similitud coseno

In [40]:
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import check_array
from scipy.spatial.distance import cosine

classifier = NearestNeighbors(radius=55)
X = results_df["feature_vector"].to_numpy()
X = np.array([item for item in X])

In [41]:
classifier.fit(X)

NearestNeighbors(radius=55)

## Generamos las predicciones y guardamos las publicidades similares en carpetas individuales

In [42]:
def filter_differents(distance, nbrs, threshold):
    new_nbrs = []
    for d, n in zip(distance, nbrs):
        if d<threshold:
            new_nbrs.append(n)
    return new_nbrs

In [43]:
classes = []
classes_img_names = []
images_already_classified = []
for counter, feature_vector in enumerate(X):
    if counter not in images_already_classified:
        rng = classifier.radius_neighbors([feature_vector], return_distance=True, sort_results=True)
        # nbrs = filter_differents(distance[0], nbrs, 30)
        # print(nbrs, distance)
        # print(np.asarray(rng[0][0]))
        nbrs = np.asarray(rng[1][0])
        classes.append(nbrs)
        tmp = []
        for c in nbrs:
            images_already_classified.append(c)
            tmp.append(results_df.iloc[c,:].src_image)
        classes_img_names.append(tmp)

In [51]:
images_already_classified.count(2)

5

In [44]:
classes_img_names[0]

['0.jpg',
 '278.jpg',
 '143.jpg',
 '287.jpg',
 '62.jpg',
 '318.jpg',
 '117.jpg',
 '57.jpg',
 '287.jpg',
 '0.jpg',
 '214.jpg',
 '310.jpg',
 '310.jpg',
 '45.jpg',
 '259.jpg',
 '263.jpg',
 '288.jpg',
 '32.jpg',
 '318.jpg',
 '63.jpg',
 '117.jpg',
 '144.jpg',
 '311.jpg',
 '6.jpg',
 '58.jpg',
 '190.jpg',
 '282.jpg',
 '152.jpg',
 '1.jpg',
 '152.jpg',
 '270.jpg',
 '314.jpg',
 '96.jpg',
 '56.jpg',
 '87.jpg',
 '143.jpg',
 '247.jpg',
 '148.jpg',
 '151.jpg',
 '24.jpg',
 '319.jpg']

In [45]:
for counter, src_images in enumerate(classes_img_names):
    imgs = [plt.imread(f"{root}/{src_image}") for src_image in src_images]
    stacked = np.hstack(imgs)
    try:
        plt.imsave(f"../../results/clustered_onlyVGG16BGR/{counter}.jpg", stacked)
    except:
        continue